In [ ]:
import numpy as np

#Define POMDP parameters
s = ['Heads','Tails']  # Heads or Tails
a = ['Predict Heads', 'Predict Tails', 'Observe'] # Predict Heads, Predict Tails, Observe
o = ['Observed Heads', 'Observed Tails']  # Observed Heads, Observed Tails

b_0 = [0.5,0.5] # initial state distribution

gamma = 0.9
# P(s'/s,a)
state_transition_model = {
    # probability of transition given the state Heads and actions
    (s[0],a[0]): [0.5,0.5],
    (s[0],a[1]): [0.5,0.5],
    (s[0],a[2]): [0.5,0.5],
    # probability of transition given the state Tails and actions
    (s[1],a[0]): [0.5,0.5],
    (s[1],a[1]): [0.5,0.5],
    (s[1],a[2]): [0.5,0.5]
}

# Define reward model R(s,a)
rewards = np.array([[10, -1, 0],  # Given Heads: Predicted Heads, Predicted Tails, Observe
                    [-1, 10, 0]])  # Given Tails: Predicted Heads, Predicted Tails, Observe
# O(o/s',a)
observation_model = {
     # probability of observation given the action Predict Heads and resulting state 
    (a[0], s[0]): [0.9,0.1], # [ O(o1/a[0], s[0]), O(o2/a[0], s[0]) ]
    (a[0], s[1]): [0.1,0.9],
     # probability of observation given the action Predict Tails and resulting state 
    (a[1], s[0]): [0.1,0.9],
    (a[1], s[1]): [0.9,0.1],
     # probability of observation given the action Observe and resulting state 
    (a[2], s[0]): [0.7,0.3],
    (a[2], s[1]): [0.3,0.7]
} 

In [6]:
# Define finite-state controller 
q = ['Start', 'PredictHeads', 'PredictTails']
action_selection_model = {
    q[0]: [1/3, 1/3, 1/3],  # Uniform distribution over actions
    q[1]: [0.8, 0.1, 0.1],  # Prefer Predict Heads, but can choose any action
    q[2]: [0.1, 0.8, 0.1]  # Prefer Predict Tails, but can choose any action
}
node_transition_model = {
    # From 'Start' ²node
    (q[0], a[0], o[0]): [0.1, 0.8, 0.1],  
    (q[0], a[0], o[1]): [0.1, 0.1, 0.8],  
    (q[0], a[1], o[0]): [0.1, 0.8, 0.1],  
    (q[0], a[1], o[1]): [0.1, 0.1, 0.8],  
    (q[0], a[2], o[0]): [0.1, 0.8, 0.1],  
    (q[0], a[2], o[1]): [0.1, 0.1, 0.8],    
    # From 'PredictHeads' node
    (q[1], a[0], o[0]): [0.1, 0.8, 0.1],  
    (q[1], a[0], o[1]): [0.1, 0.1, 0.8],  
    (q[1], a[1], o[0]): [0.1, 0.8, 0.1],  
    (q[1], a[1], o[1]): [0.1, 0.1, 0.8],  
    (q[1], a[2], o[0]): [0.1, 0.8, 0.1],  
    (q[1], a[2], o[1]): [0.1, 0.1, 0.8],   
    
    # From 'PredictTails' node
    (q[2], a[0], o[0]): [0.1, 0.8, 0.1],  
    (q[2], a[0], o[1]): [0.1, 0.1, 0.8],  
    (q[2], a[1], o[0]): [0.1, 0.8, 0.1],  
    (q[2], a[1], o[1]): [0.1, 0.1, 0.8],  
    (q[2], a[2], o[0]): [0.1, 0.8, 0.1],  
    (q[2], a[2], o[1]): [0.1, 0.1, 0.8]
}

In [7]:
def calculate_value(qu, state):
    max_value = float('-inf')
    for action in a:
        v = action_selection_model[qu][a.index(action)] * rewards[s.index(state)][a.index(action)]
        print(v)
        for sprime in s:
            for obs in o:
                for qprime in q:
                    v += gamma * \
                    action_selection_model[qu][a.index(action)] * \
                    (state_transition_model[(state,action)][s.index(sprime)] * \
                    observation_model[(action,sprime)][o.index(obs)] * \
                    node_transition_model[(qu,action,obs)][q.index(qprime)] * \
                    calculate_value(qprime, sprime))
                    print(v)
            #max_value = max(max_value, value)
    return max_value

In [22]:
def objective(q0=0):
    b0 = [1/3,1/3,1/3]
    obj = 0
    for i in range(len(s)):
        obj += b0[i] * value_function[i][0]
    returny * obj
    

In [39]:
value_function = np.random.rand(len(s),len(q))
value_function
var = [[0.5,0.5],value_function]
var[1][1][1]

0.5353279804694298

In [30]:
from scipy.optimize import minimize

# Define the objective function
q0 = 0
def objective(variables):
    b0 = [1/3,1/3,1/3]
    obj = 0
    for i in range(len(s)):
        obj += b0[i] * variables[0][i][q0]
    return -1 * obj
# Define the constraint
def constraint(x):
    return x[0]**2 + x[1]**2 - 1

# Initial guess
initial_guess = [0, 0]

# Solve the QCLP
result = minimize(objective, initial_guess, constraints={'type': 'ineq', 'fun': constraint})

# Access the solution
optimal_solution = result.x
optimal_value = result.fun

# Print the results
print("Optimal Solution:", optimal_solution)
print("Optimal Value:", optimal_value)

TypeError: object of type 'int' has no len()

In [26]:
def check_value_const(state,qu):
    value = value_function[s.index(state)][q.index(qu)]
    for action in a:
        v = action_selection_model[qu][a.index(action)] * rewards[s.index(state)][a.index(action)]
        for sprime in s:
            for obs in o:
                for qprime in q:
                    v += gamma * action_selection_model[qu][a.index(action)] * (state_transition_model[(state,action)][s.index(sprime)] * observation_model[(action,sprime)][o.index(obs)] * node_transition_model[(qu,action,obs)][q.index(qprime)] * value_function[s.index(sprime)][q.index(qprime)])
    return np.equal(value,v)

In [27]:
import numpy as np

# Define your transition probabilities P(a|q), P(o|s, a), and P(q'|q, a, o)
# Replace these with your actual probabilities
P_a_given_q = np.array([[0.8, 0.1, 0.1],
                        [0.1, 0.8, 0.1],
                        [0.1, 0.1, 0.8]])

P_o_given_s_and_a = np.array([[0.8, 0.1],
                              [0.1, 0.8]])

P_q_prime_given_q_a_o = np.array([[[0.8, 0.1, 0.1],
                                  [0.1, 0.8, 0.1],
                                  [0.1, 0.1, 0.8]],
                                 
                                 [[0.1, 0.8, 0.1],
                                  [0.8, 0.1, 0.1],
                                  [0.1, 0.1, 0.8]],
                                 
                                 [[0.1, 0.1, 0.8],
                                  [0.1, 0.1, 0.8],
                                  [0.8, 0.1, 0.1]]])

# Define your reward function R(s, a, q, q')
# Replace these with your actual rewards
R = np.array([[[-1, 0, 0],
               [0, -1, 0],
               [0, 0, -1]],
              
              [[-1, 0, 0],
               [0, -1, 0],
               [0, 0, -1]]])

# Define the discount factor
gamma = 0.9

# Define the number of FSC nodes and states
num_nodes = 3
num_states = 2

# Define a function to calculate the value of FSC node q at state s
def calculate_value(q, s):
    max_value = float('-inf')
    for a in range(len(P_a_given_q)):
        for o in range(len(P_o_given_s_and_a[a])):
            value = sum(P_a_given_q[q, a] * P_o_given_s_and_a[a, s, o] * P_q_prime_given_q_a_o[q, a, o, q_prime] * (R[s, a, q, q_prime] + gamma * calculate_value(q_prime, s))
                        for q_prime in range(num_nodes))
            max_value = max(max_value, value)
    return max_value

# Calculate the value of each FSC node at each state
values = np.zeros((num_nodes, num_states))
for q in range(num_nodes):
    for s in range(num_states):
        values[q, s] = calculate_value(q, s)

print("Value Function:")
print(values)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
import numpy as np

# Define the number of states and actions
num_states = 5
num_actions = 3

# Create a random reward matrix (state-action pairs)
rewards = np.random.rand(num_states, num_actions)

# Create a random transition probability matrix (state-action pairs)
transition_probs = np.random.rand(num_states, num_actions, num_states)
transition_probs /= np.sum(transition_probs, axis=2, keepdims=True)  # Normalize probabilities

# Set discount factor
gamma = 0.9

# Initialize value function randomly
value_function = np.random.rand(num_states)

# Perform value iteration to calculate the value function
num_iterations = 1000
i = 0
for _ in range(num_iterations):
    new_value_function = np.zeros(num_states)
    for state in range(num_states):
        expected_values = np.zeros(num_actions)
        for action in range(num_actions):
            expected_rewards = rewards[state, action]
            for next_state in range(num_states):
                expected_rewards += gamma * transition_probs[state, action, next_state] * value_function[next_state]
            expected_values[action] = expected_rewards
        new_value_function[state] = np.max(expected_values)
    value_function = new_value_function

print("Calculated Value Function:", value_function)
print(i)


Calculated Value Function: [6.58751735 7.1929661  7.04051373 6.75865885 6.93376245]
75000
